In [1]:
#Imports

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
# When importing nltk for the first time, we need to also download a few built-in libraries
!pip install nltk
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords 
from nltk import word_tokenize

stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zacharie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/zacharie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zacharie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/zacharie/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#Language Processing Functions

import string

def text_to_list(text):
    text = text.replace("['", "")
    text = text.replace("']", "")
    my_list = text.split("', '")
    return my_list

def remove_punctuation(text):
    for punctuation in string.punctuation: 
        text = text.replace(punctuation, ' ') 
    return text

def remove_nonalpha(text):
    list_of_words = text.split(' ')
    list_clean = []
    for words in list_of_words:
        words_only = ''.join([i for i in words if i.isalpha()])
        list_clean.append(words_only)
    final_words = ' '.join(list_clean)
    return final_words

def lowercase(text): 
    lowercased = text.lower() 
    return lowercased

def remove_stopwords(text):
    tokenized = word_tokenize(text)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

from nltk.stem import WordNetLemmatizer

def lemma(text):
    lemmatizer = WordNetLemmatizer() # Initiate lemmatizer
    lemmatized = [lemmatizer.lemmatize(word) for word in text] # Lemmatize
    lemmatized_string = " ".join(lemmatized)
    return lemmatized_string

def pun_list(liste):
    my_list = [remove_punctuation(text) for text in liste]
    return my_list
def num_list(liste):
    my_list = [remove_nonalpha(text) for text in liste]
    return my_list
def low_list(liste):
    my_list = [lowercase(text) for text in liste]
    return my_list
def sto_list(liste):
    my_list = [remove_stopwords(text) for text in liste]
    return my_list
def lem_list(liste):
    my_list = [lemma(text) for text in liste]
    return my_list
    

In [5]:
#data import

%load_ext autoreload
%autoreload 2

data = pd.read_csv("../raw_data/Recipes/Food Ingredients and Recipe Dataset with Image Name Mapping.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
data['Title_bis'] = data['Title'] 

In [33]:
data['Title_bis'] = data.Title_bis.apply(str)

In [37]:
data['Title_bis'] = data.Title_bis.apply(remove_punctuation)
data['Title_bis'] = data.Title_bis.apply(remove_nonalpha)
data['Title_bis'] = data.Title_bis.apply(lowercase)
data['Title_bis'] = data.Title_bis.apply(remove_stopwords)
data['Title_bis'] = data.Title_bis.apply(lemma)

In [44]:
for i in range(len(data['Title_bis'])):
    data['Title_bis'][i] = data['Title_bis'][i].split()

/var/folders/vp/c987_zkx4hg6k0yfsdxsvmd80000gn/T/ipykernel_68539/2180487437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Title_bis'][i] = data['Title_bis'][i].split()


In [51]:
#building ingredients table

all_ingredients =[]

for i in range (len(data['Title_bis'])):
    for j in range (len(data['Title_bis'][i])):
        all_ingredients.append(data['Title_bis'][i][j])

all_ingredients_sentence = ' '.join([i for i in all_ingredients])       
list_ingredients = all_ingredients_sentence.split(" ")

df = pd.DataFrame(data = list_ingredients, columns =['word'])
df['count'] = 1
ingredients_df = pd.DataFrame(df.groupby('word')['count'].sum())
ingredients_df = ingredients_df.sort_values(by='count', ascending=False)

In [55]:
from pathlib import Path  
filepath = Path('../raw_data/ingredients_title_zach.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
ingredients_df.to_csv(filepath)  


In [61]:
df_base = pd.read_csv("../raw_data/test.csv")
df_base['category'].apply(str)

In [83]:
missing_ingredients = { 
'fish' : [],
'meat' : [],
'vegetable' : [],
'fruits' : [],
'other' : [] }

In [120]:
data['Missing_Ingredient'] = np.empty((len(data), 0)).tolist()
data['Missing_Ingredient_Category'] = np.empty((len(data), 0)).tolist()

In [93]:
for key in missing_ingredients:
    for i in range(len(df_base['word'])):
        if key == df_base['category'][i]:
            missing_ingredients[key].append(df_base['word'][i])


In [129]:
for key in missing_ingredients:
    for ingredient in missing_ingredients[key]:
        for i in range(len(data['Title_bis'])):
            if ingredient in data['Title_bis'][i]:
                data['Missing_Ingredient'][i].append(ingredient)
                data['Missing_Ingredient_Category'][i].append(key)

In [137]:
data['Missing_Ingredient_Category'] = data.Missing_Ingredient_Category.apply(lambda y: np.nan if len(y)==0 else y)
data['Missing_Ingredient'] = data.Missing_Ingredient.apply(lambda y: np.nan if len(y)==0 else y)

In [152]:
data_to_export = data[['Missing_Ingredient','Missing_Ingredient_Category']]



In [151]:
data

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Title_bis,Missing_Ingredient,Missing_Ingredient_Category
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","[miso, butter, roast, chicken, acorn, squash, ...",[squash],[vegetable]
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...","[crispy, salt, pepper, potato]",NaN,NaN
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...","[thanksgiving, mac, cheese]",NaN,NaN
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...","[italian, sausage, bread, stuffing]",[sausage],[meat]
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","[newton, law]",NaN,NaN
...,...,...,...,...,...,...,...,...,...
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten...","[brownie, pudding, cake]",NaN,NaN
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s...","[israeli, couscous, roasted, butternut, squash...","[squash, couscous]","[vegetable, other]"
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...,"[rice, soy, glazed, bonito, flake, sesame, seed]",NaN,NaN
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,[spanakopita],NaN,NaN


In [153]:
data_to_export

,Missing_Ingredient,Missing_Ingredient_Category
0,[squash],[vegetable]
1,NaN,NaN
2,NaN,NaN
3,[sausage],[meat]
4,NaN,NaN
...,...,...
13496,NaN,NaN
13497,"[squash, couscous]","[vegetable, other]"
13498,NaN,NaN
13499,NaN,NaN


In [154]:
from pathlib import Path  
filepath = Path('../raw_data/missing_ingredients.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data_to_export.to_csv(filepath)  

In [145]:
missing_ingredients

{'fish': ['salmon',
  'fish',
  'crab',
  'scallop',
  'tuna',
  'cod',
  'anchovy',
  'halibut',
  'sea',
  'bass',
  'mussel',
  'oyster',
  'trout',
  'squid',
  'seafood'],
 'meat': ['pork',
  'lamb',
  'turkey',
  'bacon',
  'sausage',
  'ham',
  'prosciutto',
  'chorizo',
  'veal'],
 'vegetable': ['squash',
  'pea',
  'pumpkin',
  'beet',
  'cucumber',
  'kale',
  'zucchini',
  'asparagus',
  'eggplant',
  'cabbage',
  'broccoli',
  'celery',
  'leek',
  'artichoke',
  'polenta',
  'radish',
  'lentil',
  'brussels',
  'horseradish',
  'tofu',
  'radicchio',
  'lettuce',
  'truffle',
  'escarole',
  'kimchi',
  'collard',
  'turnip',
  'endive'],
 'fruits': ['orange',
  'coconut',
  'cherry',
  'cranberry',
  'pear',
  'peach',
  'raspberry',
  'avocado',
  'pineapple',
  'apricot',
  'fig',
  'mango',
  'pomegranate',
  'blackberry',
  'rhubarb',
  'grape',
  'raisin',
  'grapefruit',
  'watermelon',
  'lobster',
  'melon'],
 'other': ['pizza', 'couscous', 'margarita', 'champagn